In [ ]:
!pip install requests beautifulsoup4 pandas matplotlib

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import math
import time
import csv


In [ ]:
BASE_URL = "http://books.toscrape.com/catalogue/page-{}.html"
START_PAGE = 1
END_PAGE = 100
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
headers = {"User-Agent": USER_AGENT}

In [ ]:
page = requests.get("http://books.toscrape.com/catalogue/page-1.html")
def main(page):
    src = page.content
    soup = BeautifulSoup(page.content, 'html.parser')
    print(soup.prettify()[:1000])
    return soup

main(page)

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')
books = soup.find_all('article', class_='product_pod')

for book in books:
    title = book.find('h3').find('a').get('title')
    price = book.find('div', class_='product_price').find('p', class_='price_color').text
    rating = book.find('p').get('class')[1]

    print(title, price, rating)


In [ ]:


all_books = []
all_titles = []
all_prices = []
all_ratings = []

for page_number in range(START_PAGE, END_PAGE + 1):
    page_url = BASE_URL.format(page_number)
    print(f"Loading page {page_number} ...")

    page = requests.get(page_url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    books = soup.find_all('article', class_='product_pod')


    for book in books:
        title = book.find('h3').find('a').get('title')
        price =  book.find('div', class_='product_price').find('p', class_='price_color').text
        rating = book.find('p').get('class')[1]

        all_titles.append(title)
        all_prices.append(price)
        all_ratings.append(rating)
        all_books.append({
            'title': title,
            'price': price,
            'rating': rating
        })

    time.sleep(1)

print("✅ Data successfully collected from all pages!")

In [ ]:
all_books = []
df = pd.DataFrame(all_books)


In [ ]:
df = pd.DataFrame({
    'Title': all_titles,
    'Price': all_prices,
    'Rating': all_ratings
})

df.head()

In [ ]:


df['Price'] = df['Price'].str.replace('£', '').astype(float)


In [ ]:
print(df.describe())
print(df['Rating'].value_counts())


In [ ]:
avg_prices = df.groupby('Rating')['Price'].mean()
avg_prices.plot(kind='bar', title='Average Book Price by Rating')
plt.xlabel('Rating')
plt.ylabel('Average Price (£)')
plt.show()



In [ ]:
df.head()


In [ ]:
free_books = df[df['Price'] == 0]
free_books



In [ ]:
df.sort_values(by='Price').head(10)


In [ ]:
df.describe()

In [ ]:
df.to_csv('books_dataset.csv', index=False)
print("📁 Data saved to books_dataset.csv")